In [1]:
# import the necessary libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup

To build up on the sample code given, I'll traverse one row vertically, and then move on to the next column.

Otherwise, the final data frame will only contain the postal codes from the first column of the table off of the Wikipedia page.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
presoup = requests.get(url).text
soup = BeautifulSoup(presoup)

table_contents = []
table = soup.find('table')
for row in table.findAll('tr'):
    for col in row.findAll('td'):
        cell = {}
        if col.span.text == 'Not assigned':
            pass
        else:
            cell['Postal Code'] = col.p.text[:3]
            cell['Borough'] = col.p.span.text.split('(')[0]
            cell['Neighborhood'] = (((((col.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
            table_contents.append(cell)

In [3]:
# print(table_contents)
df = pd.DataFrame(table_contents)

df['Borough'] = df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

I'll take the intersection between my neighborhood data frame and the data frame read from the lat-lon CSV.

In [4]:
latslons = pd.read_csv('./Geospatial_Coordinates.csv')

df1 = df.merge(latslons, how = 'inner', indicator = False)

display(df1.head())

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [7]:
df1.head(20)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [5]:
df1.shape

(103, 5)